In [3]:
import os
import sys
import glob
import numpy as np
import torch
import utils
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
# from model import NetworkCIFAR as Network
from model_search import Network


CIFAR_CLASSES = 2

# manual loading
class Arg:
#     def __init__(self,data,batch_size,report_freq,gpu,C,layers,model_path,auxiliary,cutout,cutout_length,drop_path_prob,seed,arch):
    def __init__(self):
        self.data = '..'
        self.batch_size = 4
        self.report_freq = 50
        self.gpu = 0
        self.init_channels = 16
        self.layers = 8
        self.model_path = 'EXP/weights.pt'
        self.auxiliary = False
        self.cutout = False
        self.cutout_length = 16
        self.drop_path_prob = 0.3
        self.seed = 0
        self.arch = 'DARTS'
        self.train_portion = 0.5
CIFAR_CLASSES = 2
args = Arg()
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)

In [4]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
model = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model1 = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model = model.cuda()
model1 = model1.cuda()
train_data, val_data = utils._get_train_EEG_data(args)
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(args.train_portion * num_train))

train_queue = torch.utils.data.DataLoader(
  train_data, batch_size=args.batch_size,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
  pin_memory=False, num_workers=4)

external_queue = torch.utils.data.DataLoader(
  train_data, batch_size=args.batch_size,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
  pin_memory=False, num_workers=4)

valid_queue = torch.utils.data.DataLoader(
  train_data, batch_size=args.batch_size,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(
      indices[split:num_train]),
  pin_memory=False, num_workers=4)

test_data = utils._get_test_EEG_data(args)

test_queue = torch.utils.data.DataLoader(test_data, batch_size=args.batch_size, 
                                         shuffle=False, pin_memory=True, num_workers=2)

X_train shape: (71300, 24, 256, 1)
Y_train shape: (71300, 1)
X_val shape: (39868, 24, 256, 1)
Y_val shape: (39868, 1)
X_test shape: (16006, 24, 256, 1)
Y_test shape: (16006, 1)


In [5]:
model.load_state_dict(torch.load('EXP/weights.pt'))
model1.load_state_dict(torch.load('EXP/weights1.pt'))

<All keys matched successfully>

In [6]:
objs = utils.AvgrageMeter()
top1 = utils.AvgrageMeter()
objs1 = utils.AvgrageMeter()
top1_1 = utils.AvgrageMeter()
model.eval()
model1.eval()
with torch.no_grad():
    for step, (input, target) in enumerate(test_queue):
        input = input.cuda()
        target = target.cuda(non_blocking=True)

        logits = model(input)
        loss = criterion(logits, target)

        # prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
        prec1 = utils.accuracy(logits, target)
        n = input.size(0)
        objs.update(loss.item(), n)
        top1.update(prec1[0], n)
        # top5.update(prec5.item(), n)

        # for the second model.
        logits = model1(input)
        loss = criterion(logits, target)

        # prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
        prec1 = utils.accuracy(logits, target)
        objs1.update(loss.item(), n)
        top1_1.update(prec1[0], n)
        # top5_1.update(prec5.item(), n)
        if step % args.report_freq == 0:
            # logging.info('valid 1st %03d %e %f %f', step,
            #              objs.avg, top1.avg, top5.avg)
            # logging.info('valid 2nd %03d %e %f %f', step,
            #              objs1.avg, top1_1.avg, top5_1.avg)
            print('valid 1st %03d %e %f' % (step,
                       objs.avg, top1.avg))
            print('valid 2nd %03d %e %f' % (step,
                       objs1.avg, top1_1.avg))
    print('acc %f acc1 %f', top1.avg, top1_1.avg)

valid 1st 000 0.000000e+00 100.000000
valid 2nd 000 0.000000e+00 100.000000
valid 1st 050 9.595140e+01 74.509804
valid 2nd 050 3.245779e+01 85.294121
valid 1st 100 4.986711e+01 86.138611
valid 2nd 100 1.999328e+01 90.841583
valid 1st 150 7.266305e+01 85.430466
valid 2nd 150 2.661648e+01 87.582779
valid 1st 200 4.023153e+04 78.109451
valid 2nd 200 4.212853e+03 79.975121
valid 1st 250 3.231696e+04 74.800797
valid 2nd 250 3.401210e+03 76.394424
valid 1st 300 2.711746e+04 70.847176
valid 2nd 300 2.884264e+03 72.093025
valid 1st 350 2.329754e+04 70.655273
valid 2nd 350 2.487824e+03 71.866096
valid 1st 400 2.050200e+04 68.516212
valid 2nd 400 2.207546e+03 69.763092
valid 1st 450 1.824073e+04 69.789360
valid 2nd 450 1.967974e+03 71.008865
valid 1st 500 1.647593e+04 69.011978
valid 2nd 500 1.785788e+03 70.359283
valid 1st 550 1.498543e+04 70.916519
valid 2nd 550 1.625431e+03 72.232307
valid 1st 600 1.377561e+04 70.216301
valid 2nd 600 1.500913e+03 71.505821
valid 1st 650 1.272382e+04 69.393242

In [7]:
print('acc %f acc1 %f', top1.avg, top1_1.avg)

acc %f acc1 %f tensor(71.4545, device='cuda:0') tensor(73.0726, device='cuda:0')
